In [3]:
import pandas as pd
data = pd.read_csv("/kaggle/input/train-shuttle/train.csv")

In [4]:
data.columns

Index(['rally', 'ball_round', 'time', 'frame_num', 'roundscore_A',
       'roundscore_B', 'player', 'type', 'aroundhead', 'backhand',
       'landing_height', 'landing_area', 'landing_x', 'landing_y',
       'lose_reason', 'getpoint_player', 'player_location_area',
       'player_location_x', 'player_location_y', 'opponent_location_area',
       'opponent_location_x', 'opponent_location_y', 'set', 'match_id',
       'rally_id', 'rally_length'],
      dtype='object')

In [78]:
def filter(data,player1,player1_loc):
    out_col =[
        'rally', 'ball_round', 'time', 'frame_num', 'roundscore_A',
        'roundscore_B', 'player', 'type', 'aroundhead', 'backhand',
        'landing_height', 'landing_area', 'landing_x', 'landing_y',
        'lose_reason', 'getpoint_player', 'player_location_area',
        'player_location_x', 'player_location_y', 'opponent_location_area',
        'opponent_location_x', 'opponent_location_y', 'set', 'match_id',
        'rally_id', 'rally_length'
     ]
    output = pd.DataFrame(columns=out_col)     
    for i in range(len(data)):
        if data["player"][i]==player1 and data["player_location_area"][i]==player1_loc:
            output.loc[len(output), output.columns] = data.iloc[i] 
            #output= output.append(data.iloc[i])            
    
    return output


In [79]:
filter(data,1,1)

,rally,ball_round,time,frame_num,roundscore_A,roundscore_B,player,type,aroundhead,backhand,...,player_location_area,player_location_x,player_location_y,opponent_location_area,opponent_location_x,opponent_location_y,set,match_id,rally_id,rally_length
0,1,6,00:06:05,10966,1,0,1,net shot,0.0,0,...,1,259.8,573.61,8,240.6,263.58,1,1,0,8
1,9,20,00:10:03,18106,3,6,1,net shot,0.0,0,...,1,275.0,563.06,8,217.7,256.23,1,1,8,24
2,12,8,00:11:40,21014,5,7,1,lob,0.0,0,...,1,262.8,566.64,8,228.1,355.2,1,1,11,13
3,22,9,00:18:51,33942,10,12,1,push/rush,0.0,0,...,1,264.6,587.3,8,216.5,230.28,1,1,21,9
4,28,10,00:24:00,43213,13,15,1,lob,0.0,0,...,1,278.2,588.92,8,222.9,313.14,1,1,27,11
5,36,4,00:29:20,52805,19,17,1,lob,0.0,0,...,1,270.7,580.44,8,223.4,336.79,1,1,35,12


In [83]:
import numpy as np
import random
def augmen(data,playera, playerb,rally_id,match_id,se,scorea,scoreb):
    rally_len=1
    loc = random.randint(1,10)
    out_col =[
        'rally', 'ball_round', 'time', 'frame_num', 'roundscore_A',
        'roundscore_B', 'player', 'type', 'aroundhead', 'backhand',
        'landing_height', 'landing_area', 'landing_x', 'landing_y',
        'lose_reason', 'getpoint_player', 'player_location_area',
        'player_location_x', 'player_location_y', 'opponent_location_area',
        'opponent_location_x', 'opponent_location_y', 'set', 'match_id',
        'rally_id', 'rally_length'
     ]
    output = pd.DataFrame(columns=out_col)
    
    while True:
        if rally_len%2 :
            player = playera           
        else:
            player = playerb            
        filt = filter(data,player,loc)
        sel = random.randint(0,len(filt)-1)
        shoot = filt.iloc[sel]
        shoot["rally"] = rally_id+1
        shoot["ball_round"] = rally_len
        shoot["player"] = player
        loc = shoot["opponent_location_area"]
        shoot["set"] = se
        shoot["match_id"] = match_id
        shoot["rally_id"] = rally_id
        
        if len(str(shoot["lose_reason"]))>3:
            if shoot["getpoint_player"]!=player:
                if player == playera:
                    shoot["getpoint_player"]=playerb
                else:
                    shoot["getpoint_player"]=playera
            output.loc[len(output), output.columns] = shoot
            output = output.assign(rally_length=rally_len)
            #output = output.append(shoot)
            break
        rally_len+=1
        output.loc[len(output), output.columns] = shoot 
        #output = output.append(shoot)
    return output
    

In [84]:
playera = 0
playerb = 1
rally_id = 0
match_id = 0
se = 0
scorea = 0
scoreb = 0
augmen(data,playera,playerb,rally_id,match_id,se,scorea,scoreb)

,rally,ball_round,time,frame_num,roundscore_A,roundscore_B,player,type,aroundhead,backhand,...,player_location_area,player_location_x,player_location_y,opponent_location_area,opponent_location_x,opponent_location_y,set,match_id,rally_id,rally_length
0,1,1,00:23:11,41733,15,15,0,net shot,0.0,0,...,7,130.7,591.43,8,164.3,334.13,0,0,0,8
1,1,2,00:35:51,64546,3,1,1,net shot,0.0,0,...,8,123.0,314.31,8,151.4,668.01,0,0,0,8
2,1,3,00:34:48,62641,3,1,0,smash,0.0,0,...,8,187.7,210.57,8,192.1,708.9,0,0,0,8
3,1,4,00:40:07,72233,8,5,1,defensive shot,0.0,1,...,8,168.9,238.79,5,269.5,748.89,0,0,0,8
4,1,5,00:44:20,79821,13,10,0,net shot,0.0,0,...,5,92.2,355.96,8,136.7,670.71,0,0,0,8
5,1,6,00:09:51,17745,3,6,1,defensive shot,0.0,1,...,8,176.9,664.87,3,78.5,197.36,0,0,0,8
6,1,7,00:28:16,50881,10,7,0,drop,0.0,0,...,3,253.3,789.58,8,195.6,305.49,0,0,0,8
7,1,8,00:09:24,16924,2,6,1,smash,0.0,0,...,8,154.6,739.7,8,182.0,256.69,0,0,0,8
